In [3]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from PIL import Image, ImageTk
import librosa
import numpy as np
import joblib
from keras.models import load_model
from sklearn.preprocessing import StandardScaler

# Load the encoder and scaler
encoder = joblib.load('encoder.joblib')
scaler = joblib.load('scaler.joblib')

# Load the trained model
model = load_model('speech_model.h5')

# Function to preprocess audio file and make prediction
def extract_features(data, sample_rate):
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=np.abs(librosa.stft(data)), sr=sample_rate).T, axis=0)
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    return np.hstack((zcr, chroma_stft, mfcc, rms, mel))

def predict_emotion(file_path):
    try:
        data, sample_rate = librosa.load(file_path, duration=2.5, offset=0.6)
        features = extract_features(data, sample_rate)
        features_scaled = scaler.transform(np.expand_dims(features, axis=0))
        features_scaled = np.expand_dims(features_scaled, axis=2)
        prediction = model.predict(features_scaled)
        emotion_label = encoder.categories_[0][np.argmax(prediction)]
        return emotion_label
    except Exception as e:
        print("Error:", e)
        return "Error"

# Function to handle file selection
def browse_file():
    file_path = filedialog.askopenfilename(filetypes=[("Audio Files", "*.wav")])
    if file_path:
        emotion_label = predict_emotion(file_path)
        result_label.config(text=f"Predicted Emotion: {emotion_label}")

# main application window
app = tk.Tk()
app.title("Emotion Speech Prediction")
app.geometry("400x200") 

# widgets
browse_button = tk.Button(app, text="Browse", command=browse_file)
result_label = tk.Label(app, text="Predicted Emotion: ")

# Pack widgets
browse_button.pack(pady=20)
result_label.pack()

# Start the GUI application
app.mainloop()


C:\Users\dell\anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\dell\anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


1/1 [==============================] - 0s 297ms/step
